### Project#3 Mikito, Hicham, Linan

### Mikito

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
import folium


In [2]:
data = pd.read_csv("NYPD_Motor_Vehicle_Collisions.csv")

FileNotFoundError: [Errno 2] File b'NYPD_Motor_Vehicle_Collisions.csv' does not exist: b'NYPD_Motor_Vehicle_Collisions.csv'

In [ ]:
interesting_columns = ['DATE','TIME','BOROUGH','LATITUDE', 'LONGITUDE',
        'ON STREET NAME', 'CROSS STREET NAME','NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED']

In [ ]:
data.dtypes

In [ ]:
data = data[interesting_columns]

In [ ]:
data.head()

In [ ]:
danger = data.groupby(["ON STREET NAME", "CROSS STREET NAME"])["NUMBER OF PERSONS INJURED"].sum()

In [ ]:
danger.sort_values().tail(6)

In [ ]:
data[(data["NUMBER OF PEDESTRIANS INJURED"]>=1)|(data["NUMBER OF CYCLIST INJURED"]>=1)]

### 5 top dangerous intersections in NYC (on street, cross street name and # of injured people)

In [ ]:

# Make a data frame with dots to show on the map
data2 = pd.DataFrame({
'lat':[40.801830,40.663080, 40.658650, 40.716080, 40.640653],
'lon':[-73.931400,-73.962390,-73.890610,-73.803140,  -73.743340],
'name':['East 125th Street, 1 Ave ','Flatbush Ave, Empire Boulevard', 'Pennsylvania Ave, Linden Boulvard', 
        '164th Street, Grand Central Parkway', 'Rockaway Boulevard, Brookville Boulevard'],
'injured people':['94', '95', '96', '115', '191']})
 
# Make an empty map
map1 = folium.Map(location=[40.7348,-73.9060], tiles="OpenStreetMap", zoom_start=11)

# I can add marker one by one on the map
for i in range(0,5):
    folium.Marker([data2.iloc[i]['lat'], data2.iloc[i]['lon']], tooltip=data2.iloc[i]['name']+'<br> Injured people: '+data2.iloc[i]['injured people']).add_to(map1)

folium.TileLayer('Stamen Terrain').add_to(map1)
folium.LayerControl().add_to(map1)


map1.save("dangerous_intersections.html")


In [ ]:
from folium import plugins
from folium.plugins import HeatMap

In [ ]:
def get_hour(s):
    if type(s) == str:
        return s[0:2].replace(":", "")
    else:
        return s;

In [ ]:
data.head()

In [ ]:
data["hour"] = data["TIME"].apply(get_hour) #making new column

In [ ]:
data.head(10000)

In [ ]:
data_Oct_2018= data[(data["DATE"].str.startswith("10/")) & (data["DATE"].str.endswith("2018"))]

### Heatmap: Motor Vehicle Collisions in NY October, 2018

In [ ]:
map2 = folium.Map(location=[40.7508,-73.9060],
                    zoom_start = 11) 
data_Oct_2018 = data_Oct_2018[["hour","LATITUDE","LONGITUDE"]].dropna()
heat_df = data_Oct_2018[['LATITUDE', 'LONGITUDE']]

heat_df["Weight"] = data_Oct_2018["hour"].dropna()
heat_df['Weight'] = heat_df['Weight'].astype(float)
heat_df = heat_df.dropna(axis=0, subset=['LATITUDE','LONGITUDE', "Weight"])

heat_data = [[[row['LATITUDE'],row['LONGITUDE']] for index, row in heat_df[heat_df['Weight'] == i].iterrows()] for i in range(0,24)]
hm = plugins.HeatMapWithTime(heat_data,radius = 13,auto_play=False,max_opacity=0.8)
hm.add_to(map2)
map2

### Hicham

In [ ]:
import seaborn as sns
import cufflinks as cf
import plotly
import plotly.offline as py
import plotly.graph_objs as go
cf.go_offline() # required to use plotly offline (no account required).
py.init_notebook_mode() # graphs charts inline (IPython)

In [ ]:
nyc=pd.read_csv('NYPD_Motor_Vehicle_Collisions.csv')
nyc.head()

In [ ]:
nyc.DATE = pd.to_datetime(nyc.DATE)
nyc.DATE.sort_values().index
df_by_date = nyc.iloc[nyc.DATE.sort_values().index]

In [ ]:
collisions_by_date = df_by_date.groupby('DATE').DATE.count()
collisions_by_date.iplot(kind='scatter', title='Collisions Per Day')

deaths_by_date = df_by_date.groupby('DATE')['NUMBER OF PERSONS KILLED'].sum()
deaths_by_date.iplot(kind='bar', title='Deaths per Day')

In [ ]:
colli_deaths = pd.DataFrame({'collisions':collisions_by_date, 'deaths':deaths_by_date })

color1 = '#9467bd'
color2 = '#F08B00'

trace1 = go.Scatter(
    x = colli_deaths.index,
    y = colli_deaths['collisions'],
    name='collisions',
    line = dict(
        color = color1
    )
)
trace2 = go.Scatter(
    x= colli_deaths.index,
    y =colli_deaths['deaths'] ,
    name='deaths',
    yaxis='y2',
    mode='markers'

)
data = [trace1, trace2]
layout = go.Layout(
    title= "Collisions and Deaths per day",
    yaxis=dict(
        title='collisions',
        titlefont=dict(
            color=color1
        ),
        tickfont=dict(
            color=color1
        )
    ),
    yaxis2=dict(
        title='deaths',
        overlaying='y',
        side='right',
        titlefont=dict(
            color=color2
        ),
        tickfont=dict(
            color=color2
        )

    )

)
fig = go.Figure(data=data, layout=layout)
plot_url = py.iplot(fig)

In [ ]:
plt.figure(figsize=(16,9))
plt.scatter(nyc.LONGITUDE, nyc.LATITUDE, alpha=0.05, s=1, color='darkseagreen')
plt.title('Motor Vehicle Collisions in New York City', size=25)
plt.xlim((-74.26,-73.7))
plt.ylim((40.5,40.92))
plt.xlabel('Longitude',size=20)
plt.ylabel('Latitude',size=20)

plt.show()

In [ ]:
nyc=pd.read_csv('NYPD_Motor_Vehicle_Collisions.csv',parse_dates=[['DATE','TIME']])

In [ ]:
df_by_date = nyc.sort_values(['DATE_TIME'])

In [ ]:
df_by_date['month'] = df_by_date['DATE_TIME'].dt.month
df_by_date['hour'] = df_by_date['DATE_TIME'].dt.hour
df_by_date['year'] = df_by_date['DATE_TIME'].dt.year

In [ ]:
plt.style.use('ggplot')
collisions_by_date = df_by_date.groupby('hour').size()
plt.title('Collisions per Hour')
collisions_by_date.plot.bar()
plt.xticks(rotation=360)
plt.show()

In [ ]:
plt.style.use('ggplot')
collisions_by_date = df_by_date.groupby('month').size()
plt.title('Collisions per Month')
plt.xticks(rotation=90)
collisions_by_date.plot.bar()
plt.xticks(rotation=360)
plt.show()

In [ ]:
plt.style.use('ggplot')
collisions_by_date = df_by_date.groupby('year').size()
plt.title('Collisions per Year')
plt.xticks(rotation=45)
collisions_by_date.plot.bar()
plt.xticks(rotation=360)
plt.show()

In [ ]:
month_hour = df_by_date[['month','hour']]

In [ ]:
by_borough = nyc.groupby('BOROUGH').sum()
by_borough.reset_index(inplace=True)

In [ ]:
plt.style.use('ggplot')
by_borough.sort_values('NUMBER OF PERSONS KILLED').plot.barh(x='BOROUGH',y='NUMBER OF PERSONS KILLED')
plt.title('Number of people killed by Borough')

### Linan

In [ ]:
data = pd.read_csv("NYPD_Motor_Vehicle_Collisions.csv")

In [ ]:
interesting_columns = ['DATE','TIME','BOROUGH','LATITUDE', 'LONGITUDE',
        'ON STREET NAME', 'CROSS STREET NAME','NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED']

In [ ]:
data = data[interesting_columns]
data

In [ ]:
danger = data.groupby(["BOROUGH"])["NUMBER OF PERSONS INJURED"].sum()
danger

In [ ]:
danger = data.groupby(["BOROUGH"])["NUMBER OF PERSONS KILLED"].sum()
danger

In [ ]:
danger = data.groupby(["BOROUGH"])["NUMBER OF PEDESTRIANS INJURED"].sum()
danger

In [ ]:
danger = data.groupby(["BOROUGH"])["NUMBER OF PEDESTRIANS KILLED"].sum()
danger

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111,polar=True)
#values in ratio
values_a = [37720/87621,148/339,9216/19831,85/195,37720/87621]
values_b = [87621/87621,339/339,19831/19831,195/195,87621/87621]
values_c = [41341/87621,195/339,14444/19831,146/195,41341/87621]
values_d=  [66888/87621,318/339,13478/19831,181/195,66888/87621]
values_e=  [10633/87621,59/339,1613/19831,24/195,10633/87621]


angles = np.linspace(0,2*np.pi,len(values_a))
categories = ['INJURED', 'Killed', 'PEDESTRIANS INJURED', 'PEDESTRIANS KILLED']

#Graph for each brough
ax.plot(angles,values_a,linewidth=6,label='Bronx')
ax.fill(angles,values_a,alpha=.3)

ax.plot(angles,values_b,linewidth=6,label="Brooklyn")
ax.fill(angles,values_b,alpha=.3)

ax.plot(angles,values_c,linewidth=6,label="Manthattan")
ax.fill(angles,values_c,alpha=.3)

ax.plot(angles,values_d,linewidth=6,label="Queens")
ax.fill(angles,values_d,alpha=.3)

ax.plot(angles,values_e,linewidth=6,label="STATEN ISLAND")
ax.fill(angles,values_e,alpha=.3)

ax.set_title('Report comparsion of each Brough Collisions')
ax.set_xticks(angles)

 
ax.set_xticklabels(categories)
ax.legend()

plt.show()